# TP3: Integration numerique
### realise par : 
####                 Bacem Abroug 2DNI2
________
## Intro:
En analyse numérique, il existe une vaste famille d’algorithmes dont le but principal est d’estimer la valeur numérique de l’intégrale définie sur un domaine particulier pour une fonction donnée (par exemple l’intégrale d’une fonction d’une variable sur un intervalle).

Ces techniques procèdent en trois phases distinctes :

    +Décomposition du domaine en morceaux (un intervalle en sous-intervalles contigus)

    +Intégration approchée de la fonction sur chaque morceau 

    +Sommation des résultats numériques ainsi obtenus.
_______________

In [1]:

%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from numpy import *

## Généralités:
Considérons une intégrale définie $${\displaystyle I=\int _{a}^{b}f(x)\,\mathrm {d} x}$$ dont on cherche à estimer la valeur numérique.
### Les moyens les plus simples sont :
**1.la méthode des rectangles à gauche :**
____________
on approche ${\displaystyle I=\int _{a}^{b}f(x)\,\mathrm {d} x=(b-a)f(a)}$  par . Géométriquement, cela signifie qu'on approche l'intégrale de f par l'aire des rectangles hachurés en vert :
![](http://www.bibmath.net/dico/i/images/intnum4.png)


In [142]:
class RectangleG ( object ) :
    def __init__ (self , a , b , n , f ) :
        self.a = a
        self.b = b
        self.x = np.linspace( a , b , n+1 )
        self.f = f
        self.n = n
    def integrate ( self , f ) :
        x= self.x
        y= [f( xx ) for xx in x]
        h = float( x[1] - x[0] )
        s = sum( y[ 0 : -1 ] )
        return h * s
    def Graph ( self , f , resolution =1001 ) :
        xl = self.x
        yl =[f(x) for x in xl]
        xlist_fine =np.linspace( self.a , self.b , resolution )
        for i in range ( self.n ) :
            x_rect = [xl[ i ] , xl[ i ] , xl[ i + 1 ] , xl[i+1] , xl[ i ] ] # abscisses des sommets
            y_rect = [0 , yl[ i ] , yl[ i ] , 0 , 0 ] # ordonnees des sommets
            plt.plot ( x_rect , y_rect , 'r' )
        yflist_fine = [f ( x ) for x in xlist_fine]
        plt.plot ( xlist_fine , yflist_fine )
        plt.plot(xl, yl,"bo")
        plt.xlabel ( 'x' )
        plt.ylabel ( ' f ( x ) ' )
        plt.title ( ' Methode des rectangles gauches' )
        plt.text( 0.5*( self.a+ self.b ) , f(self.b ) , 'I_{} ={:0.8f}'.format(self.n,self.integrate( f ) ) , fontsize =15 )

**2.Formule du trapèze:**
_____________________
En interpolant f par un polynôme de degré 1, les deux points d'interpolation (a, f (a)) et (b, f (b)) suffisent à tracer un segment dont l’intégrale correspond à l’aire d’un trapèze, justifiant le nom de méthode des trapèzes qui est d’ordre 1 :

$${\displaystyle I(f)=(b-a)\,{\frac {f(a)+f(b)}{2}}}$$
L’erreur est donnée par

$${\displaystyle E(f)=-{\frac {(b-a)^{3}}{12}}f''(\eta ),\quad \eta \in [a,b]}$$
Conformément aux expressions de l’erreur, la méthode des trapèzes est souvent moins performante que celle du point milieu.
![](https://www.researchgate.net/profile/Nour_Moustafa4/publication/317558878/figure/fig5/AS:669621539590159@1536661624308/Composite-trapezoidal-rule.png)

In [143]:
class Trapezoidal(object):
    def __init__(self, a, b, n, f):
        self.a = a
        self.b = b
        self.x = np.linspace(a, b, n+1)
        self.f = f
        self.n = n
    def integrate(self,f):
        x=self.x
        y= [f( xx ) for xx in x]
        h = float(x[1] - x[0])
        s = y[0] + y[-1] + 2.0*sum(y[1:-1])
        return h * s / 2.0
    def Graph(self,f,resolution=1001):
        xl = self.x
        yl = [f(x) for x in xl]
        xlist_fine=np.linspace(self.a, self.b, resolution)
        for i in range(self.n):
            x_rect = [xl[i], xl[i], xl[i+1], xl[i+1], xl[i]] # abscisses des sommets
            y_rect = [0   , yl[i], yl[i+1]  , 0     , 0   ] # ordonnees des sommets
            plt.plot(x_rect, y_rect,"m")
        yflist_fine = [f ( x ) for x in xlist_fine]
        plt.plot(xlist_fine, yflist_fine)#plot de f(x)
        plt.plot(xl, yl,"cs")#point support
        plt.ylabel ( ' f ( x ) ' )
        plt.title ( ' Methode des Trapèzes' )
        plt.text( 0.5*( self.a+ self.b ) , f(self.b ) , 'I_{} ={:0.8f}'.format(self.n,self.integrate( f ) ) , fontsize =15 )

**3.Formule de Simpson**
________________
En interpolant f par un polynôme de degré 2 (3 degrés de liberté), 3 points (ou conditions) sont nécessaires pour le caractériser : les valeurs aux extrémités a, b, et celle choisie en leur milieu m = (a + b) / 2. La méthode de Simpson est basée sur un polynôme de degré 2 (intégrale d’une parabole), tout en restant exacte pour des polynômes de degré 3 ; elle est donc d’ordre 3 :

$${\displaystyle I(f)={\frac {(b-a)}{6}}\left[f(a)+4f\left({\frac {a+b}{2}}\right)+f(b)\right]}{\displaystyle I(f)={\frac {(b-a)}{6}}\left[f(a)+4f\left({\frac {a+b}{2}}\right)+f(b)\right]}$$
L’erreur globale est donnée par

$${\displaystyle E(f)=-{\frac {(b-a)^{5}}{2880}}f^{(4)}(\eta ),\quad \eta \in [a,b]}E(f)=-{\frac  {(b-a)^{5}}{2880}}f^{{(4)}}(\eta ),\quad \eta \in [a,b]$$
Remarque : comme la méthode du point milieu qui caractérise un polynôme de degré 0 et qui reste exacte pour tout polynôme de degré 1, la méthode de Simpson caractérise un polynôme de degré 2 et reste exacte pour tout polynôme de degré 3. Il s’agit d’une sorte d’anomalie où se produisent des compensations bénéfiques à l’ordre de la méthode.
![](https://www.intmath.com/integration/img/simpsons-6a.png)

In [144]:

class Simpson(object):
    def __init__(self, a, b, n, f): #initialiser les paramètres du classe
        self.a = a
        self.b = b
        self.x = np.linspace(a, b, n+1)#les pts supports
        self.f = f
        self.n = n #nombre de subdivision

    def integrate(self,f):#calculer la somme ((b-a)/6*n)*[f(a)+2*sum(xi)+4*sum(mi)+f(b)]
        x=self.x #les points supports xi #x(0)=a-->x(n)=b
        y= [f( xx ) for xx in x] #yi variable local y(o)=f(xo)-->y(n)
        h = float(x[1] - x[0])#pas h=(b-a)/2*n
        n = len(x) - 1#nombre subdivision
        if n % 2 == 1:#si le reste de la division =1 impaire
            n -= 1#☺nombre de sub ywali paire
        s = y[0] + y[n] + 4.0 * sum(y[1:-1:2]) + 2.0 * sum(y[2:-2:2])
        #y[1:-1:2] min impaire loulla m0 lil 9bal likhrania 5ater 3anna deja y(n) par pas de 2== mi
        #calculer la somme
        #T(-1] dernier valeur dans le tableau)
        return h * s / 3.0
    def Graph(self,f,resolution=1001):#1000 points 1001 résolution juste pour dessiner f
        xl = self.x #pt support
        yl = [f(x) for x in xl] #yi
        xlist_fine=np.linspace(self.a, self.b, resolution)
        # pour le graph de la fonction f #intervalle ab subdiviser en 1000 poitns
        for i in range(self.n):#range intervalle 0 à n
            xx=np.linspace(xl[i], xl[i+1], resolution)
            #pour chaque subdivisuion  on doit dessiner polynome dnc on doit aussi le subdiviser
            m=(xl[i]+xl[i+1])/2#pt milieu
            aa=xl[i]#borne gauche
            bb=xl[i+1]#borne droite
            l0 = (xx-m)/(aa-m)*(xx-bb)/(aa-bb)
            l1 = (xx-aa)/(m-aa)*(xx-bb)/(m-bb)
            l2 = (xx-aa)/(bb-aa)*(xx-m)/(bb-m)
            P = f(aa)*l0 + f(m)*l1 + f(bb)*l2#fonction dde polynome
            plt.plot(xx,P,'b')#dessiner polynome d'interpolation
            plt.plot(m,f(m),"r*")
        yflist_fine = [f ( x ) for x in xlist_fine]#fontion f
        plt.plot(xlist_fine, yflist_fine,'g')
        plt.plot(xl, yl,'bo')#point support en bleu rond
        
        plt.ylabel('f(x)')
        plt.title('Simpson')
        plt.text( 0.5*( self.a+ self.b ) , f(self.b ) , 'I_{} ={:0.8f}'.format(self.n,self.integrate( f ) ) , fontsize =15 )

**4.Formules du rectangle et du point milieu:**
_____________________
C’est la méthode la plus simple qui consiste à interpoler la fonction f à intégrer par une fonction constante (polynôme de degré 0).

Si ξ est le point d’interpolation, la formule est la suivante :

${\displaystyle I(f)=(b-a)f(\xi )\,}I(f)=(b-a)f(\xi )\$,
Le choix de ξ influence l’erreur E(f) = I – I(f) :

Si ξ = a ou ξ = b, l’erreur est donnée par
$${\displaystyle E(f)={\frac {(b-a)^{2}}{2}}f'(\eta ),\quad \eta \in [a,b].}E(f)={\frac  {(b-a)^{2}}{2}}f'(\eta ),\quad \eta \in [a,b].$$
C’est la méthode du rectangle qui est d’ordre 0.
Si ξ = (a + b)/2, l’erreur est donnée par
$${\displaystyle E(f)={\frac {(b-a)^{3}}{24}}f''(\eta ),\quad \eta \in [a,b].}E(f)={\frac  {(b-a)^{3}}{24}}f''(\eta ),\quad \eta \in [a,b].$$
Il s’agit de la méthode du point milieu qui est d’ordre 1.
Ainsi, le choix du point milieu améliore l’ordre de la méthode : celle du rectangle est exacte (c’est-à-dire E(f) = 0) pour les fonctions constantes alors que celle du point milieu est exacte pour les polynômes de degré 1. Ceci s’explique par le fait que l’écart d’intégration de la méthode du point milieu donne lieu à deux erreurs d’évaluation, de valeurs absolues égales et de signes opposés.
![](http://www.bibmath.net/dico/i/images/intnum8.png)

In [145]:
class Milieu(object): #class rectange 
    def __init__(self, a, b, n, f):#initialiser les paramètres du classe
        self.a = a
        self.b = b
        self.x = np.linspace(a, b, n+1)
        self.f = f
        self.n = n
    def integrate(self,f):
        x=self.x# contiens les xi
        h = float(x[1] - x[0])
        s=0
        for i in range(self.n):
            s=s+f((x[i]+x[i+1])*0.5)
        return h*s
       
    def Graph(self,f,resolution=1001):
        xl = self.x
        yl = [f(x) for x in xl]
        xlist_fine=np.linspace(self.a, self.b, resolution)        
        for i in range(self.n):            
            m=(xl[i]+xl[i+1])/2
            x_rect = [xl[i], xl[i], xl[i+1], xl[i+1], xl[i]] # abscisses des sommets
            y_rect = [0   , f(m), f(m)  , 0     , 0   ] # ordonnees des sommets
            plt.plot(x_rect, y_rect,"r")
            plt.plot(m,f(m),"b*")
            
        yflist_fine = [f ( x ) for x in xlist_fine]
        plt.plot(xlist_fine, yflist_fine,'g')
        plt.xlabel('x')
        plt.ylabel('f(x)')
        plt.title('Milieu')
        
        plt.text( 0.5*( self.a+ self.b ) , f(self.b ) , 'I_{} ={:0.8f}'.format(self.n,self.integrate( f ) ) , fontsize =15 )

**4.la méthode des rectangles à droite : **
_______________________
on approche ${\displaystyle I=\int _{a}^{b}f(x)\,\mathrm {d} x=(b-a)f(b)}$  par . Géométriquement, cela signifie qu'on approche l'intégrale de f par l'aire des rectangles hachurés en rouge :
![](http://www.bibmath.net/dico/i/images/intnum6.png)

In [146]:
class RectangleD ( object ) :
    def __init__ (self , a , b , n , f ) :
        self.a = a
        self.b = b
        self.x = np.linspace( a , b , n+1 )
        self.f = f
        self.n = n
    def integrate ( self , f ) :
        x= self.x
        y= [f( xx ) for xx in x]
        h = float( x[1] - x[0] )
        s = sum( y[ 1 :  ] )
        return h * s
    def Graph ( self , f , resolution =1001 ) :
        xl = self.x
        yl = [f(x) for x in xl]
       
        xlist_fine =np.linspace( self.a , self.b , resolution )
        for i in range ( self.n ) :
            x_rect = [xl[ i+1 ] , xl[ i ] , xl[ i] , xl[i+1] , xl[ i+1 ] ] # abscisses des sommets
            y_rect = [0 , 0, yl[ i+1 ] , yl[i+1] , 0 ] # ordonnees des sommets
            plt.plot ( x_rect , y_rect , 'r' )
        yflist_fine = [f ( x ) for x in xlist_fine]
        plt.plot ( xlist_fine , yflist_fine )
        plt.plot(xl, yl,"bo")
        plt.xlabel ( 'x' )
        plt.ylabel ( ' f ( x ) ' )
        plt.title ( ' Methode des rectangles droites' )
        plt.text( 0.5*( self.a+ self.b ) , f(self.b ) , 'I_{} ={:0.8f}'.format(self.n,self.integrate( f ) ) , fontsize =15 )

## Remarque:
Sous de bonnes hyothèses sur la fonction f (elle doit être continue dans les 3 premiers cas, C1 dans les 3 suivants), on prouve que ces méthodes d'intégration numérique fonctionnent : quand on augmente le nombre de fois où on coupe le segment [a,b], la valeur de l'intégrale approchée converge vers la vraie valeur de l'intégrale.

In [147]:
%%html
<style>
.lbl_bg{
    width:auto;
    background-color:#E8E8E8;
}
.box_style{
    width:40%;
    border : 2px solid red;
    height: auto;
    background-color:gray;
}
</style>

In [148]:

# create some control elements
Sel=widgets.Dropdown(
    options=[('Méthode des réctangles gauches', 1), ('Méthode des Trapèzes', 2),('Méthodes des Points Milieux', 3) ,('Méthodes de Simpson', 4),
             ('Méthode des réctangles droites', 5)],
    value=1,
    description='Méthode:',
)
Sel.add_class('lbl_bg')
text_func = widgets.Text(value='cos(x)', description='Fonction', continuous_update=False)
text_func.add_class('lbl_bg')
text_xlabel = widgets.Text(value='', description='xlabel', continuous_update=False)

text_ylabel = widgets.Text(value='', description='ylabel', continuous_update=False)
text_a = widgets.Text(value='-1', description='a', continuous_update=False)
text_a.add_class('lbl_bg')
text_b = widgets.Text(value='1', description='b', continuous_update=False)
text_b.add_class('lbl_bg')
text_n = widgets.Text(value='3', description='n', continuous_update=True)
text_n.add_class('lbl_bg')
text_int = widgets.Text(value='', description='I_n', continuous_update=True)
text_int.add_class('lbl_bg')
button = widgets.Button(description="Calculer",button_style='info')
button.add_class('lbl_bg')
# callback functions
def sim(b):
    
    ax.clear()
    dic={1:RectangleG,2:Trapezoidal,3:Milieu,4:Simpson,5:RectangleD}
    s=Sel.value
    plt.cla()
    func=lambda x:eval(text_func.value)
    
    R=dic[s](float(text_a.value), float(text_b.value),int(text_n.value),func)
    R.Graph(func)
    text_int.value=str(R.integrate(func))
    text_int.description='I_'+text_n.value
    displayF()



def update_a(change):
    change.new    
def update_b(change):
    change.new
def update_n(change):
    change.new
def update_f(change):
    change.new

button.on_click(sim)
# connect callbacks and traits

text_func.observe(update_f, 'value')
text_a.observe(update_a, 'value')
text_b.observe(update_b, 'value')

In [149]:

output = widgets.Output()
with output:
    fig, ax = plt.subplots( figsize=(6, 4))
    fig.canvas.toolbar_position = 'top'

def displayF():
    with output:
        
        output.clear_output(wait=True)
        
        
        ax.grid(True)
        
       
        display(fig)

        

In [150]:

def make_boxes():
    vbox1 = widgets.VBox([Sel, text_func,text_a,text_b,text_n,button,text_int])
    displayF()
    vbox2 = widgets.VBox([output])
    return vbox1, vbox2
box_layout = widgets.Layout(
        border='solid 2px gray',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px')


vbox1, vbox2 = make_boxes()
vbox1.add_class("box_style")
vbox2.add_class("box_style")
vbox1.layout = box_layout
vbox2.layout = box_layout

widgets.HBox([vbox1, vbox2])

| Nom de la Méthode | Degré du  polynôme | Nombre de  points | Degré  d’exactitude  ( ordre ) | Degré  d’erreur globale | Degré  d’erreur finale |
|-------------------|--------------------|-------------------|--------------------------------|-------------------------|------------------------|
| RectangleG        | 0                  | 1                 | 0                              | 2                       | 1                      |
| Point Milieu      | 0                  | 1                 | 1                              | 3                       | 2                      |
| Trapèze           | 1                  | 2                 | 1                              | 3                       | 2                      |
| Simpson           | 2                  | 3                 | 3                              | 5                       | 4                      |